<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_ga/blob/main/PIFWIA_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GA - Genetic Algorithm ou Algoritmo Genético é uma metaheurística derivada da Computação Evolucionária tendo como um dos primeiros pesquisadores Melanie Mitchell em 1996 que se propõe resolver problemas de otimização e busca inspirado no processo de seleção natural fazendo uso de operadores como mutação, cruzamento e seleção.

Mitchell, M. Genetic algorithms: An overview. Complexity, Volume 1, Issue 1 Pages: 4-67. https://doi.org/10.1002/cplx.6130010108, 1995.

In [ ]:
#instala biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importa bibliotecas
import random
import math
import matplotlib.pyplot as plt
from numpy.random import choice
import Orange

In [ ]:
#define os hiperparâmetros
DIMENSOES = 2 #determina a quantidade de dimensões do problema
ITERACOES = 200 #quantiddade máxima de ciclos (episódios) especificando quantas explorações podem ser realizadas
POPULACAO = 20 #tamanho da população correspondente ao número de formigas no formigueiro
E1 = -15 #extremo esquerdo eixo x
E2 = 15 #extremo direito eixo x
E3 = -100 #extremo inferior eixo y
E4 = 100 #extremo superior eixo y
LIMITES = [E3,E4] #(bound) determina os valores maximos e minimos do espaço de busca (região de busca)
FCUSTO = 'distancia' #(fitness) define a aptidão do indivíduo na possível solução do problema
TORNEIO = 0.4 # taxa do método de seleção torneio
ROLETA = 0.6 # taxa do método de seleção roleta
MUTACAO = 0.1 # taxa do operador de mutação
INDIVIDUOS = [] #(swarm) array da criação dos indivíduos da população

In [ ]:
#importa dados
from google.colab import files  
files.upload()

In [ ]:
#instancia objeto de dados com base no caminho gerado na importação do arquivo
dados = Orange.data.Table("/content/dados.csv")

In [ ]:
#explora os metadados e dados da arquivo importado
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)
dados.domain.attributes #explora os domínios dos atributos (campos da base de dados)
print("Registros:", len(dados)) #explora os dados (quantidade de registros da base de dados)
i = 0 #exibe os primeiros registros para análise dos dados importados
for d in dados[:20]:
  i += 1
  print(i, d)

In [ ]:
#cria arrays das dimensões do problema a ser otimizado
periodo = []
complexidade = [] #1-muito baixa complexidade;2-baixa complexidade;3-média complexidade;4-alta complexidade;e,5-muito alta complexidade
pagina = []
prazo = []
revisao = []
entrega = []
i = 0
for d in dados[:POPULACAO]:
  periodo.append(d[1])
  complexidade.append(d[2])
  pagina.append(d[3])
  prazo.append(d[4])
  revisao.append(d[5])
  entrega.append(d[6])
  print("id:",i,"período:",periodo[i],"complexidade:",complexidade[i],"página:",pagina[i],"prazo:",prazo[i],"revisões:",revisao[i],"entrega:",entrega[i])
  per = periodo[i]
  cpx = complexidade[i]
  pag = pagina[i]
  prz = prazo[i]
  rev = revisao[i]
  ent = entrega[i]
  i += 1

In [ ]:
#função custo ou objetivo ou aptidão ou otimização ou fitness - usada para buscar o melhor ponto dentro de um espaço de buscao (melhor global) sem ficar preso em um melhor local
def fcusto(possivel_solucao):
  fitness = 0
  for i in range(0, POPULACAO): 
    if i < POPULACAO - 1:
      fitness += pontos[possivel_solucao[i]].distancia(pontos[possivel_solucao[i+1]])
    else:
      fitness += pontos[possivel_solucao[i]].distancia(pontos[possivel_solucao[0]])
  return 1/fitness

In [ ]:
#indivíduo - unidade base da otimização, posicionada numa determinada posição no espaço de busca do problema, representando uma solução em potencial para o problema
class Individuo:
  def __init__(self, x, y):
    self.posicao_x = x
    self.posicao_y = y
  def distancia(self, individuo_outro):
    return math.sqrt((self.posicao_x - individuo_outro.posicao_x)**2 + (self.posicao_y - individuo_outro.posicao_y)**2)

In [ ]:
#mutação - operação que modifica aleatoriamente alguma característica do indivíduo criando novos valores de características em uma pequena quantidade da população
#introduzindo diversidade genética na população assegurando a probabilidade de se chegar a qualquer ponto do espaço de busca
def mutacao(individuo):
  if random.random() < MUTACAO:
    individuo_mutado = []
    aux_1 = random.randint(0, POPULACAO-1)
    aux_2 = random.randint(0, POPULACAO-1)
    while(aux_1 == aux_2):
      aux_2 = random.randint(0, POPULACAO-1)
    if aux_2 < aux_1:
      aux_3 = aux_1
      aux_1 = aux_2
      aux_2 = aux_3
    for i in range(aux_1):
      individuo_mutado.append(individuo[i])
    for i in range(aux_2-1, aux_1-1, -1):
      individuo_mutado.append(individuo[i])
    for i in range(aux_2, len(individuo)):
      individuo_mutado.append(individuo[i])
    return individuo_mutado
  return individuo

In [ ]:
#crossover - operação de cruzamento onde são criados novos indivíduos misturando características de dois indivíduos base características de um indivíduo
#são trocados pelo trecho equivalente do outro indivíduo #resultando em um novo indivíduo que potencialmente combina as melhores características dos indivíduos base
def crossover(individuos, pais):
  aux_1 = random.randint(0, POPULACAO-1)
  aux_2 = random.randint(0, POPULACAO-1)
  if aux_2 < aux_1:
    aux_3 = aux_1
    aux_1 = aux_2
    aux_2 = aux_3
  list_pai = individuos[pais[0]][aux_1:aux_2]
  filho = []
  i = 0
  while len(filho) < aux_1:
    if individuos[pais[1]][i] not in list_pai:
      filho.append(individuos[pais[1]][i])
    i+=1
  filho = filho + list_pai
  i = 0
  while len(filho) < POPULACAO:
    if individuos[pais[1]][i] not in filho:
      filho.append(individuos[pais[1]][i])
    i+=1
  return filho

In [ ]:
#selecao - operação de seleção de um indivíduo de uma população para aplicação dos operadores genéticos através de um valor aptidão podendo ser através do método "Roleta" ou "Torneio"
def selecao_base(fitness):
  pais = []
  for i in range(POPULACAO):
    pais.append(torneio(fitness))
  return pais

In [ ]:
#torneio - um número n de indivíduos da população é escolhido aleatoriamente para formar uma sub-população temporária
#deste grupo o indivíduo selecionado dependerá de uma probabilidade k definida previamente
def torneio(fitness):
  pais = []
  for k in range(0, 2):
    individuos_torneio = []
    for i in range(POPULACAO):
      aux = random.random()
      if aux < TORNEIO:
        individuos_torneio.append(i)
    while len(individuos_torneio) < 2:
      individuos_torneio.append(random.randint(0, POPULACAO-1))
    best = fitness[individuos_torneio[0]]
    best_ind = individuos_torneio[0]
    for i in range(len(individuos_torneio)):
      if fitness[individuos_torneio[i]] > best:
        best = fitness[individuos_torneio[i]]
        best_ind = individuos_torneio[i]
    pais.append(best_ind)
  return pais

In [ ]:
#roleta - cada indivíduo da população é representado proporcionalmente ao seu índice de aptidão, assim indivíduos com alta aptidão possuem uma porção maior da roleta
def roleta(fitness):
  pais = []
  for k in range(0, 2):
    individuos_roleta = []
    for i in range(POPULACAO):
      aux = random.random()
      if aux < ROLETA:
        individuos_roleta.append(i)
    while len(individuos_roleta) < 2:
      individuos_roleta.append(random.randint(0, POPULACAO-1))
    best = fitness[individuos_roleta[0]]
    best_ind = individuos_roleta[0]
    for i in range(len(individuos_roleta)):
      if fitness[individuos_roleta[i]] > best:
        best = fitness[individuos_roleta[i]]
        best_ind = individuos_roleta[i]
    pais.append(best_ind)
  return pais

In [ ]:
#calcula posições
def calcula_solucao():
  lista_individuos = list(range(POPULACAO))
  lista_aleatoria = []
  for i in range(POPULACAO):
    individuo = random.choice(lista_individuos)
    lista_aleatoria.append(individuo)
    lista_individuos.remove(individuo)
  return lista_aleatoria

In [ ]:
#plota posições no gráfico
def exibe_grafico(titulo, lista_pontos, solucao):
  x = []
  y = []
  for i in range(POPULACAO):
    x.append(lista_pontos[solucao[i]].posicao_x)
    y.append(lista_pontos[solucao[i]].posicao_y)
    plt.plot(lista_pontos[solucao[i]].posicao_x, lista_pontos[solucao[i]].posicao_y, 'ro')
  x.append(lista_pontos[solucao[0]].posicao_x)
  y.append(lista_pontos[solucao[0]].posicao_y)
  plt.axis([E1,E2,E3,E4])
  plt.plot(0,0, marker='*', markersize=10, linewidth=0.8, color='b')
  plt.plot(x,y, color='c')
  plt.title(titulo)
  plt.show()

In [ ]:
#inicializa população (indivíduos)
for i in range(POPULACAO):
  cpx = complexidade[i]
  pag = pagina[i]
  posicao = [cpx,pag]
  INDIVIDUOS.append(posicao)
  print("indivíduo:",i,"complexidade:",INDIVIDUOS[-1][0],"página:",INDIVIDUOS[-1][1])

In [ ]:
#posiciona os indivíduos no espaço de busca
pontos = []
for i in range(POPULACAO):
  pontos.append(Individuo(INDIVIDUOS[i][0], INDIVIDUOS[i][1]))
individuos = []
fitness = []
fitness_tempo = []
for i in range(POPULACAO):
  individuos.append(calcula_solucao())
  fitness.append(fcusto(individuos[-1]))
print('Distancia inicial:',1/fitness[0])
exibe_grafico('Solução aleatória inicial', pontos, individuos[0])

In [ ]:
#otimização GA
best = 0
best_route = []
for i in range(ITERACOES):
  print("Iteração: {:.0f}".format(i+1))
  pais = selecao_base(fitness)
  filhos = []
  for j in pais:
    filhos.append(crossover(individuos,j))
  individuos = filhos
  for k in range(POPULACAO):
    individuos[k] = mutacao(individuos[k])
  fitness = []
  for k in range(POPULACAO):
    fitness.append(fcusto(individuos[k]))
    if fitness[-1] > best:
      best = fitness[-1]
      best_route = individuos[k]
  exibe_grafico('Melhor rota ' + str(best_route), pontos, best_route)
  fitness_tempo.append(1/best)

In [ ]:
print('Melhor:',1/best)
exibe_grafico('Melhor Solução', pontos, best_route)

In [ ]:
#exibe a curva de convergência dos indivíduos
x = []
y = []
for i in range(len(fitness_tempo)):
  x.append(i)
  y.append(fitness_tempo[i])
plt.plot(x, y)
plt.title('Desempenho X Tempo')
plt.show()

In [ ]:
#exibe das posições e otimizações calculadas
posicao_inicial = []
posicao_otimizada = []
for i in range(POPULACAO):
  posicao_inicial.append(INDIVIDUOS[i])
  posicao_otimizada.append(INDIVIDUOS[i])
  print("i:",i,"posição:",posicao_inicial[i],"otimização:",posicao_otimizada[i])